In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import tensorflow as tf

from keras.metrics import Precision, Recall, AUC, CategoricalAccuracy
from sklearn.model_selection import GroupShuffleSplit, StratifiedShuffleSplit

from custom_models.cnns import simple_cnn_bn
from etl.load_dataset import DatasetProcessor, get_tf_eggim_patch_dataset
from evaluation.metrics import categorical_accuracy, sensitivity_per_class, specificity_per_class

In [16]:
num_folds = 5
target_dir = '../test_files/EGGIMazing/Dataset'
dp = DatasetProcessor(target_dir)
df = dp.process()
def replace_landmark_name(landmark):
    new_landmark_name = {'ii': 'distal body',
                    'ix': 'distal lc',
                    'vi': 'proximal ant lc',
                    'vii': 'proximal ant gc/pw',
                    'viii': 'incisura r',
                    'x': 'upper body r',
                    'xii': 'upper body ant'}
    landmark_number = landmark.split('.')[0]
    return new_landmark_name[landmark_number]
df['landmark'] = df['landmark'].apply(replace_landmark_name)
togas_ids_boolean = np.array([x.startswith('PT') for x in df['patient_id'].values])
df_togas = df[togas_ids_boolean].reset_index(drop=True)
df_ipo = df[~togas_ids_boolean].reset_index(drop=True)

Threshold de decisão EGGIM = 5
 EGGIM = EGGIM score vi. proximal antrum - lesser curvature + EGGIM score vii. proximal antrum - greater curvature, posterior wall + EGGIM score viii. incisura in retroflexion + [(EGGIM score ii. distal body in anteversion + EGGIM score xii. upper body in anteversion)/2] + [(EGGIM score ix. distal lesser curvature in retroflexion + EGGIM score x. upper body in retroflexion)/2]

In [135]:


def compute_eggim_patient(patient_df, target_variable):
    eggim_landmarks = ['proximal ant lc',
     'proximal ant gc/pw',
     'incisura r',
     'distal body',
     'upper body ant',
     'distal lc',
     'upper body r']

    df = patient_df[['landmark', target_variable]].value_counts()
    # we use index[0] to assume first landmark/score pair is correct
    eggim_antrum_incisura = df['proximal ant lc'].index[0] + df['proximal ant gc/pw'].index[0] + df['incisura r'].index[0]
    eggim_body_1 = (df['distal body'].index[0] + df['upper body ant'].index[0]) / 2
    eggim_body_2 = (df['distal lc'].index[0]+ df['upper body r'].index[0]) / 2
    return eggim_antrum_incisura + eggim_body_1 + eggim_body_2



def get_eggim_df(df):
    eggim_scores = {}
    patient_ids = np.unique(df['patient_id'].values)
    for i in range(len(patient_ids)):
        try:
            eggim_square = compute_eggim_patient(df[df.patient_id == patient_ids[i]],
                                          target_variable='eggim_square')
            eggim_global = compute_eggim_patient(df[df.patient_id == patient_ids[i]],
                                          target_variable='eggim_global')
            eggim_scores[patient_ids[i]] = {'eggim_square': eggim_square, 'eggim_global': eggim_global}
        except:
            continue
    return eggim_scores
# 2, 3, 4
pd.DataFrame(get_eggim_df(df_togas)).T

,eggim_square,eggim_global
PT004,1.0,1.0
PT005,4.0,3.0
PT006,1.0,1.0
PT009,8.0,8.5
PT016,0.0,0.0
PT020,1.0,2.0
PT021,10.0,10.0
PT022,8.0,8.0


landmark            eggim_square
distal body         0               2
distal lc           0               2
incisura r          1               1
                    2               1
proximal ant gc/pw  0               1
proximal ant lc     0               1
                    2               1
upper body ant      0               1
upper body r        0               1
Name: count, dtype: int64


1.0

In [56]:
df_togas[df_togas.patient_id == patient_ids[4]]

,bbox,eggim_global,eggim_square,landmark,image_directory,patient_id
32,"[341, 202, 565, 426]",1,1,incisura r,../test_files/EGGIMazing/Dataset/2024012416234...,PT006
33,"[209, 137, 433, 361]",0,0,distal body,../test_files/EGGIMazing/Dataset/2024012416241...,PT006
34,"[222, 215, 446, 439]",0,0,upper body r,../test_files/EGGIMazing/Dataset/2024012416243...,PT006
35,"[220, 234, 444, 458]",1,0,distal lc,../test_files/EGGIMazing/Dataset/2024012416250...,PT006
36,"[222, 179, 446, 403]",0,0,upper body ant,../test_files/EGGIMazing/Dataset/2024012416252...,PT006
37,"[284, 165, 508, 389]",0,0,distal body,../test_files/EGGIMazing/Dataset/2024012416263...,PT006
38,"[183, 108, 407, 332]",0,0,distal lc,../test_files/EGGIMazing/Dataset/2024012416271...,PT006
39,"[250, 101, 475, 326]",0,0,incisura r,../test_files/EGGIMazing/Dataset/2024012416280...,PT006
40,"[256, 140, 481, 364]",1,1,proximal ant gc/pw,../test_files/EGGIMazing/Dataset/2024012416284...,PT006
41,"[240, 138, 464, 362]",0,0,proximal ant lc,../test_files/EGGIMazing/Dataset/2024012416291...,PT006


In [59]:
df_togas[df_togas.patient_id == patient_ids[0]]

,bbox,eggim_global,eggim_square,landmark,image_directory,patient_id
0,"[218, 113, 442, 338]",0,0,distal body,../test_files/EGGIMazing/Dataset/2024011716000...,PT002
1,"[213, 228, 438, 452]",0,0,distal lc,../test_files/EGGIMazing/Dataset/2024011716033...,PT002
2,"[190, 231, 415, 455]",0,0,distal lc,../test_files/EGGIMazing/Dataset/2024011716040...,PT002
3,"[169, 234, 393, 458]",1,0,upper body r,../test_files/EGGIMazing/Dataset/2024011716043...,PT002
4,"[232, 129, 456, 353]",0,0,distal body,../test_files/EGGIMazing/Dataset/2024011716074...,PT002
5,"[222, 133, 446, 358]",0,0,distal lc,../test_files/EGGIMazing/Dataset/2024011716081...,PT002
6,"[207, 125, 431, 349]",2,2,proximal ant lc,../test_files/EGGIMazing/Dataset/2024011716114...,PT002


In [76]:
d_ = df_togas[df_togas.patient_id == patient_ids[2]][['landmark', 'eggim_square']].value_counts()
d_

landmark            eggim_square
distal body         0               2
distal lc           0               2
incisura r          1               1
                    2               1
proximal ant gc/pw  0               1
proximal ant lc     0               1
                    2               1
upper body ant      0               1
upper body r        0               1
Name: count, dtype: int64

In [103]:
d_['incisura r'].index[0]

eggim_square
1    1
2    1
Name: count, dtype: int64

In [105]:
d_['proximal ant lc'].index[1]

2